In [1]:
# General import 

import numpy as np
import scipy.sparse as sparse
from scipy.integrate import ode
import time
import matplotlib.pyplot as plt
from pyMPC.mpc import MPCController
import dynamics
import sympy as sp
import numpy as np
from sympy.physics.vector import dynamicsymbols
from sympy.physics.vector import time_derivative
from sympy.physics.vector import ReferenceFrame
N = ReferenceFrame('N')
import pylab as pl
import control
from EoM import *
from sympy.physics.mechanics import *
from numpy.linalg import matrix_rank, eig
import math
import intelligent_robotics as ir


In [2]:
x_next = dynamics.get_x_next()

T_w1, T_w2 = sp.symbols('T_w1, T_w2')
theta_P, theta_w1, theta_w2 = dynamicsymbols('theta_P, theta_w1, theta_w2')

In [3]:
# System dynamics: \dot x = f_ODE(t,x,u)
# x = [[theta_1], [theta_2], [theta_p], [dtheta_1], [dtheta_2], [dtheta_p]]^T

def f_ODE(x_next,x,u):
    T_w1 = u[0]
    T_w2 = u[1]
    
    param = {theta_P:x[2], theta_P.diff():x[5]}
    x_next_1 = msubs(x_next,param)
    
    dtheta_w1 = x[3]
    dtheta_w2 = x[4]
    dtheta_P = x[5]
    
    der = np.zeros(6)
    der[0] = dtheta_w1
    der[1] = dtheta_w2
    der[2] = dtheta_P
    der[3] = x_next_1[3]
    der[4] = x_next_1[4]
    der[5] = x_next_1[5]
    
    return der

In [4]:
Ac = np.array([[0,0,0,1,0,0],
              [0,0,0,0,1,0],
              [0,0,0,0,0,1],
              [0,0,-25.03210719,0,0,0],
              [0,0,-25.03210719,0,0,0],
              [0,0,29.1477018,0,0,0]])

Bc = np.array([[0,0],
              [0,0],
              [0,0],
              [1.6579922,0.00699069],
              [0.00699069,1.6579922],
              [-0.46024303, -0.46024303]])

In [5]:
# Linearized System Matrices 
    
# Continuous-time system matrices, linearized about the upright, unstable equilibrium point 
[nx, nu] = Bc.shape # number of states and number or inputs


# Simple forward euler discretization
Ts = 50e-3
Ad = np.eye(nx) + Ac*Ts
Bd = Bc*Ts

In [6]:
# MPC reference input and states (set-points)

xref = np.array([0.2, 0.2, 0, 0, 0, 0]) # reference state
uref = np.array([0, 0])    # reference input
uminus1 = np.array([0, 0])     # input at time step negative one - used to penalize the first delta u at time instant 0. Could be the same as uref.

# Constraints
xmin = np.array([-100, -100, -100, -100, -100, -100])
xmax = np.array([ 100,  100,  100,  100,  100,  100])

umin = np.array([-40, -40])
umax = np.array([40, 40])

# Constraints input variation with respect to previous sample
Dumin = np.array([-5, -5])
Dumax = np.array([5, 5])

In [7]:
# MPC cost function weights

Qx = sparse.diags([0.5, 0.5, 1, 0, 0, 0])   # Quadratic cost for states x0, x1, ..., x_N-1
QxN = sparse.diags([0.5, 0.5, 1, 0, 0, 0])  # Quadratic cost for xN
Qu = 0.0 * sparse.eye(2)        # Quadratic cost for u0, u1, ...., u_N-1
QDu = 0.01 * sparse.eye(2)       # Quadratic cost for Du0, Du1, ...., Du_N-1

In [8]:
# Initialize simulation system

phi0 = 15*2*np.pi/360
x0 = np.array([0, 0, phi0, 0, 0, 0]) # initial state
t0 = 0
system_dyn = ode(f_ODE).set_integrator('vode', method='bdf')
system_dyn.set_initial_value(x0, t0)
system_dyn.set_f_params(0.0)

In [9]:
Np = 50
# Initialize and setup MPC controller

t1 = time.time()
K = MPCController(Ad,Bd,Np=Np, x0=x0,xref=xref,uminus1=uminus1,
                  Qx=Qx, QxN=QxN, Qu=Qu,QDu=QDu,
                  xmin=xmin,xmax=xmax,umin=umin,umax=umax,Dumin=Dumin,Dumax=Dumax)
t3 = time.time()
K.setup() # this initializes the QP problem for the first step

uMPC = uminus1
K.update(system_dyn.y, uMPC) # update with measurement
uMPC = K.output() # MPC step (u_k value)
t2 = time.time()
print(t2-t1)
print(t2-t3)
print(t3-t1)

0.050076961517333984
0.049065589904785156
0.0010113716125488281


In [10]:
system_dyn.y

array([0.        , 0.        , 0.26179939, 0.        , 0.        ,
       0.        ])

In [11]:
uMPC = uminus1
K.update(system_dyn.y, uMPC) # update with measurement
uMPC = K.output() # MPC step (u_k value)
uMPC

array([5.00405219, 5.01779702])

In [13]:
 #Simulate in closed loop. Use MPC model as real system

# Simulate in closed loop
[nx, nu] = Bd.shape # number of states and number or inputs
len_sim = 10 # simulation length (s)
nsim = int(len_sim/Ts) # simulation length(timesteps)
xsim = np.zeros((nsim,nx))
usim = np.zeros((nsim,nu))
tsim = np.arange(0,nsim)*Ts

time_start = time.time()

t_step = t0
uMPC = uminus1
for i in range(nsim):
    xsim[i,:] = system_dyn.y

    # MPC update and step. Could be in just one function call
    K.update(system_dyn.y, uMPC) # update with measurement
    uMPC = K.output() # MPC step (u_k value)
    usim[i,:] = uMPC

    # System simulation
    system_dyn.set_f_params(uMPC) # set current input value
    system_dyn.integrate(t_step + Ts)

    # Time update
    t_step = t_step + Ts
time_sim = time.time() - time_start

AttributeError: 'float' object has no attribute 'args'

In [ ]:
# Plot results

fig,axes = plt.subplots(3,1, figsize=(10,10))
axes[0].plot(tsim, xsim[:,0], "k", label='p')
axes[0].plot(tsim, xref[0]*np.ones(np.shape(tsim)), "r--", label="p_ref")
axes[0].set_title("Position (m)")

axes[1].plot(tsim, xsim[:,2]*360/2/np.pi, label="phi")
axes[1].plot(tsim, xref[2]*360/2/np.pi*np.ones(np.shape(tsim)), "r--", label="phi_ref")
axes[1].set_title("Angle (deg)")

axes[2].plot(tsim, usim[:,0], label="u")
axes[2].plot(tsim, uref*np.ones(np.shape(tsim)), "r--", label="u_ref")
axes[2].set_title("Force (N)")

for ax in axes:
    ax.grid(True)
    ax.legend()